<a href="https://colab.research.google.com/github/leiyi-ihb/ColabPractics/blob/main/lyProject/03_04_quiz_modelOptimization03.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
# validation the modles found from 03script
# Solve the problem of different results showed in same model
from numpy.random import seed
seed(1)
import tensorflow
tensorflow.random.set_seed(1)  #The different tensorflow version have different code 
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, MaxPooling1D, GlobalMaxPooling1D, Conv1D, Flatten, AveragePooling1D, GlobalAveragePooling1D
from keras.optimizers import Adam, SGD

**解决03_04脚本中结果不可重复的问题：**查阅[资料](https://www.jiqixuexishequ.com/how-to-use-keras-to-get-reproducible-results.html)在脚本【03_04_quiz_modelOptimization02.ipynb】中我们在脚本的最开始两行固定了随机数种子，但结果显示问题还是存在。因此我进一步将随机数种子固定在了建模开始处，这下发现每次运行结果都一样了。  
`那我们进一步把开头固定的随机数种子去掉，又会有什么结果呢？？参阅脚本【03_04_quiz_modelOptimization04.ipynb】, 发现结果还是可重复的，说明脚本开始固定随机数没什么用`

In [8]:
! if [ ! -f "dataset_Li.pkl" ]; then wget https://master.dl.sourceforge.net/project/summer-research-20/sim_data_1/dataset_Li.pkl ; else echo "Found previous downloaded data."; fi

Found previous downloaded data.


In [9]:
# Data processing
import pickle
with open("dataset_Li.pkl", "rb") as f:
    data = pickle.load(f)
x_train, y_train = np.array(data['train'][0]), np.array(data['train'][1])
x_test, y_test = np.array(data['test'][0]), np.array(data['test'][1])

print(x_train.shape)

(16000, 1000, 4)


In [25]:

class Model:
  seed(1)
  tensorflow.random.set_seed(1)  #最开始，我只在脚本开头设置了这个，没有在这里设置，发现问题还是存在，通过在这里设置之后，所有结果就很稳定了
  def __init__(self, learnRate):        
    '''optimizer'''
    self.adam = Adam(lr=learnRate)
    self.sgd = SGD(lr=learnRate) 
    self.model = Sequential()

  def modelBuild(self, filter, poolSize, kernelSize, ConvAct, DenAct, input):
    model = self.model
    '''model build'''   
    model.add(Conv1D(filters=filter, kernel_size=(kernelSize), activation=ConvAct, input_shape=input))
    model.add(MaxPooling1D(pool_size=(poolSize)))
    model.add(GlobalMaxPooling1D())
    model.add(Flatten())
    model.add(Dense(32, activation=DenAct))
    model.add(Dense(1, activation=DenAct))
    print(model.summary())

  def modelCompile(self, met, los):
    '''model compile'''
    model = self.model
    model.compile(optimizer=self.adam,
             metrics = [met],
             loss = los)

  def modelFit(self, xTrain, yTrain, epo, batSize): 
    '''model fix'''
    model = self.model 
    model.fit(xTrain, 
              yTrain,
              epochs = epo,
              batch_size = batSize
              )
    
  def modelEvaluate(self, xTrain, yTrain, xTest, yTest, batSize):
    '''model evaluate'''
    model = self.model
    loss, accuracy = model.evaluate(xTrain, yTrain, batch_size=batSize)
    print("\ntest loss:", loss)
    print('accuracy:', accuracy)

    loss, accuracy = model.evaluate(xTest, yTest) #将我们的训练集也评估一下
    print("\ntrain loss:", loss)
    print('accuracy:', accuracy)



if __name__=='__main__':
  '''
  1, Conv1d: Filters = {32, 16, 3}; Kernel_size = {20, 10, 7, 5, 3}; Activation = {‘relu’, ‘tanh’, ‘linear’}
  2, Pooling1d: Max or Avg pool; Pool size = {8, 4, 2,}
  3, Flatten: Global Max, Global Avg, Flatten
  4, Dense: Units = {32, 64}; Activation = {‘relu’, ‘tanh’, ‘linear’}
  '''
  kernel_size =[20, 10, 7, 5, 3]
  activation = ["relu", "tanh", "linear"]
  poolSize = [8, 4, 2]
  deAct = ["relu", "tanh", "linear"]



  obj1 = Model(0.01)
  # obj1.modelBuild(filter=32, poolSize=i[2], kernelSize=i[0], ConvAct=i[1], DenAct=i[3], input=(1000, 4))
  obj1.modelBuild(filter=32, poolSize=2, kernelSize=20, ConvAct="tanh", DenAct="tanh", input=(1000, 4))
  obj1.modelCompile(met="acc", los='binary_crossentropy')
  obj1.modelFit(xTrain=x_train, yTrain=y_train, epo=10, batSize=512)
  obj1.modelEvaluate(xTrain=x_train, yTrain=y_train, xTest=x_test, yTest=y_test, batSize=512)

Model: "sequential_17"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_17 (Conv1D)          (None, 981, 32)           2592      
                                                                 
 max_pooling1d_17 (MaxPoolin  (None, 490, 32)          0         
 g1D)                                                            
                                                                 
 global_max_pooling1d_17 (Gl  (None, 32)               0         
 obalMaxPooling1D)                                               
                                                                 
 flatten_17 (Flatten)        (None, 32)                0         
                                                                 
 dense_34 (Dense)            (None, 32)                1056      
                                                                 
 dense_35 (Dense)            (None, 1)               

In [20]:

class Model:
  seed(1)
  tensorflow.random.set_seed(1)  
  def __init__(self, learnRate):        
    '''optimizer'''
    self.adam = Adam(lr=learnRate)
    self.sgd = SGD(lr=learnRate) 
    self.model = Sequential()

  def modelBuild(self, filter, poolSize, kernelSize, ConvAct, DenAct, input):
    model = self.model
    '''model build'''   
    model.add(Conv1D(filters=filter, kernel_size=(kernelSize), activation=ConvAct, input_shape=input))
    model.add(MaxPooling1D(pool_size=(poolSize)))
    model.add(GlobalMaxPooling1D())
    model.add(Flatten())
    model.add(Dense(32, activation=DenAct))
    model.add(Dense(1, activation=DenAct))
    print(model.summary())

  def modelCompile(self, met, los):
    '''model compile'''
    model = self.model
    model.compile(optimizer=self.adam,
             metrics = [met],
             loss = los)

  def modelFit(self, xTrain, yTrain, epo, batSize): 
    '''model fix'''
    model = self.model 
    model.fit(xTrain, 
              yTrain,
              epochs = epo,
              batch_size = batSize
              )
    
  def modelEvaluate(self, xTrain, yTrain, xTest, yTest, batSize):
    '''model evaluate'''
    model = self.model
    loss, accuracy = model.evaluate(xTrain, yTrain, batch_size=batSize)
    print("\ntest loss:", loss)
    print('accuracy:', accuracy)

    loss, accuracy = model.evaluate(xTest, yTest) #将我们的训练集也评估一下
    print("\ntrain loss:", loss)
    print('accuracy:', accuracy)



if __name__=='__main__':
  '''
  1, Conv1d: Filters = {32, 16, 3}; Kernel_size = {20, 10, 7, 5, 3}; Activation = {‘relu’, ‘tanh’, ‘linear’}
  2, Pooling1d: Max or Avg pool; Pool size = {8, 4, 2,}
  3, Flatten: Global Max, Global Avg, Flatten
  4, Dense: Units = {32, 64}; Activation = {‘relu’, ‘tanh’, ‘linear’}
  '''
  kernel_size =[20, 10, 7, 5, 3]
  activation = ["relu", "tanh", "linear"]
  poolSize = [8, 4, 2]
  deAct = ["relu", "tanh", "linear"]

  from itertools import product
  loop_val = [kernel_size, activation, poolSize, deAct] #
  for i in product(*loop_val):
    print(x_train.shape)
    print(i)
    if i == (20, 'tanh', 2, 'tanh'):
      obj1 = Model(0.01)
      obj1.modelBuild(filter=32, poolSize=i[2], kernelSize=i[0], ConvAct=i[1], DenAct=i[3], input=(1000, 4))
      obj1.modelCompile(met="acc", los='binary_crossentropy')
      obj1.modelFit(xTrain=x_train, yTrain=y_train, epo=10, batSize=512)
      obj1.modelEvaluate(xTrain=x_train, yTrain=y_train, xTest=x_test, yTest=y_test, batSize=512)

(16000, 1000, 4)
(20, 'relu', 8, 'relu')
(16000, 1000, 4)
(20, 'relu', 8, 'tanh')
(16000, 1000, 4)
(20, 'relu', 8, 'linear')
(16000, 1000, 4)
(20, 'relu', 4, 'relu')
(16000, 1000, 4)
(20, 'relu', 4, 'tanh')
(16000, 1000, 4)
(20, 'relu', 4, 'linear')
(16000, 1000, 4)
(20, 'relu', 2, 'relu')
(16000, 1000, 4)
(20, 'relu', 2, 'tanh')
(16000, 1000, 4)
(20, 'relu', 2, 'linear')
(16000, 1000, 4)
(20, 'tanh', 8, 'relu')
(16000, 1000, 4)
(20, 'tanh', 8, 'tanh')
(16000, 1000, 4)
(20, 'tanh', 8, 'linear')
(16000, 1000, 4)
(20, 'tanh', 4, 'relu')
(16000, 1000, 4)
(20, 'tanh', 4, 'tanh')
(16000, 1000, 4)
(20, 'tanh', 4, 'linear')
(16000, 1000, 4)
(20, 'tanh', 2, 'relu')
(16000, 1000, 4)
(20, 'tanh', 2, 'tanh')
Model: "sequential_12"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d_12 (Conv1D)          (None, 981, 32)           2592      
                                                                 
 max_p